In [19]:
import numpy as np
import cv2
import glob
from icecream import ic
from pickle import dump, load

Checkerboard and criteria for calibration

In [12]:
CHECKERBOARD = (10,7)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

Get calibration images

In [14]:
# Open the default camera
cam = cv2.VideoCapture(0)

cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280 )
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cam.set(cv2.CAP_PROP_FPS, 30)

# Get the default frame width and height
frame_width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cam.get(cv2.CAP_PROP_FPS))

ic(frame_width, frame_height, fps)

n = 0

while True:
    ret, raw = cam.read()

    # Display the captured frame
    gray = cv2.cvtColor(raw, cv2.COLOR_RGB2GRAY)
    
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)

    if ret:
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)

        # Draw and display the corners
        frame = cv2.drawChessboardCorners(raw, CHECKERBOARD, corners2, ret)
    else:
        frame = raw
     
    cv2.imshow('img',frame)
    # Press 'q' to exit the loop
    if cv2.waitKey(1) == ord('q'):
        break
    elif cv2.waitKey(1) == ord('c'):
        if ret:
            cv2.imwrite('calibration images/{}.png'.format(n), raw)
            ic("captured", n)
            n += 1

# Release the capture and writer objects
cam.release()
cv2.destroyAllWindows()

ic| frame_width: 1280, frame_height: 720, fps: 30
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'
ic| 'captured'


Calibration

In [15]:
# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []
# Creating vector to store vectors of 2D points for each checkerboard image
imgpoints = [] 

# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
objp *= 25
prev_img_shape = None
 
# Extracting path of individual image stored in a given directory
images = glob.glob('./calibration images/*.png')
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
     
    """
    If desired number of corner are detected,
    we refine the pixel coordinates and display 
    them on the images of checker board
    """
    if ret:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
         
        imgpoints.append(corners2)
 
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
     
    cv2.imshow('img',img)
    cv2.waitKey(1)
 
cv2.destroyAllWindows()
 
h,w = img.shape[:2]
 
"""
Performing camera calibration by 
passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the 
detected corners (imgpoints)
"""

'\nPerforming camera calibration by \npassing the value of known 3D points (objpoints)\nand corresponding pixel coordinates of the \ndetected corners (imgpoints)\n'

In [20]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
ic(mtx, dist)

with open('calibration_logi.calib', 'wb') as f:
    obj = (mtx,dist)
    dump(obj, f)

ic| mtx: array([[1.43067430e+03, 0.00000000e+00, 5.99982295e+02],
                [0.00000000e+00, 1.42179214e+03, 3.79029987e+02],
                [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
    dist: array([[-4.41241680e-02,  1.51145033e-01, -1.33631484e-03,
                   1.03031492e-03,  1.18314690e+00]])
